In [1]:
import cv2
import numpy as np
from PIL import Image, ImageDraw

# generate a weave pattern red and white

In [35]:
def generate_weave_pattern(width, height, num_threads_horizontal, num_threads_vertical, thread_color, background_color, thread_width, thread_distance, thread_shape, thread_layout):
    # Create a blank image with the specified width and height
    img = Image.new('RGBA', (width, height), background_color)
    draw = ImageDraw.Draw(img)

    # Calculate the gap between threads based on the distance
    horizontal_gap = thread_distance + thread_width
    vertical_gap = thread_distance + thread_width

    # Draw the horizontal threads
    for i in range(1, num_threads_horizontal):
        for j in range(num_threads_vertical):
            x_start = i * horizontal_gap
            y_start = j * vertical_gap

            if thread_layout == 'cubic_spline':
                pass
            else:
                # Draw straight thread
                draw.rectangle([(x_start, y_start), (x_start + thread_width, y_start + height)], fill=thread_color)

    # Draw the vertical threads
    for j in range(1, num_threads_vertical):
        for i in range(num_threads_horizontal):
            x_start = i * horizontal_gap
            y_start = j * vertical_gap

            if thread_layout == 'cubic_spline':
                pass
            else:
                draw.rectangle([(x_start, y_start), (x_start + width, y_start + thread_width)], fill=thread_color)

    return img


weave_pattern = generate_weave_pattern(
    width=1800,
    height=1800,
    num_threads_horizontal=6,
    num_threads_vertical=6,
    thread_color=(255, 0, 0, 255),  # here 255 makes it opaque, using 0 will make it transparent
    background_color=(255, 255, 255, 255),  # White background
    thread_width=100,
    thread_distance=200,
    thread_shape='straight',
    thread_layout='simple'
)

weave_pattern.save('result_images/weave_pattern.png')


# edge detection + dots at intersection + lines

In [36]:
def process_weave_pattern(weave_pattern):
    
    weave_cv = cv2.cvtColor(np.array(weave_pattern), cv2.COLOR_RGBA2BGR)
    gray = cv2.cvtColor(weave_cv, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(gray, (3,3), 0) 

    # Apply Canny edge detection
    edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200)

    edges_pil = Image.fromarray(edges)
    edges_pil.save('result_images/weave_edges.png')
    
    
    draw = ImageDraw.Draw(weave_pattern)
    img = Image.open('result_images/weave_pattern.png').convert('RGBA')

    thread_color = (255, 0, 0, 255)  # Thread color
    background_color = (255, 255, 255, 255)  # Background color

    thread_width_sum = 0
    thread_width_count = 0

    thread_distances = []  # Store distances between adjacent threads

    in_thread = False
    thread_start = None

    for x in range(img.width):
        thread_pixel = img.getpixel((x, 0))

        if thread_pixel == thread_color:
            if not in_thread:
                thread_start = x
                in_thread = True
        else:
            if in_thread:
                thread_end = x - 1
                thread_width = thread_end - thread_start 
                thread_width_sum += thread_width
                thread_width_count += 1

                if thread_start is not None:
                    thread_distances.append(thread_start)  # Store distance between adjacent threads

                in_thread = False
    thread_width = thread_width_sum / thread_width_count
    thread_distance = thread_distances[1] - thread_distances[0] - thread_width
    print(thread_width_count)


    # Calculate the gap between threads based on the distance
    horizontal_gap = thread_distance + thread_width
    vertical_gap = thread_distance + thread_width
    
    num_threads_horizontal=thread_width_count +1
    num_threads_vertical=thread_width_count+1

    # Draw dots at the center of intersection points
    for i in range(1, num_threads_horizontal):
        for j in range(1, num_threads_vertical):
            x_start = i * horizontal_gap
            y_start = j * vertical_gap

            draw.ellipse((x_start + thread_width // 2 - 10, y_start + thread_width // 2 - 10, x_start + thread_width // 2 + 10, y_start + thread_width // 2 + 10), fill=(0, 0, 0, 255))

    weave_pattern.save('result_images/image_with_dots.png')
    
    draw = ImageDraw.Draw(weave_pattern)
    
    # Mark the center of the vertical threads with lines
    for i in range(num_threads_horizontal):
        x_start = i * horizontal_gap
        y_start = 0
        y_end = weave_pattern.height

        center_x = x_start + thread_width // 2
        draw.line([(center_x, y_start), (center_x, y_end)], fill=(0, 0, 0), width=2)

    # Mark the center of the horizontal threads with lines
    for j in range(num_threads_vertical):
        y_start = j * vertical_gap
        x_start = 0
        x_end = weave_pattern.width

        center_y = y_start + thread_width // 2
        draw.line([(x_start, center_y), (x_end, center_y)], fill=(0, 0, 0), width=2)
        
    weave_pattern.save('result_images/image_with_lines.png')

    return edges
    
# open image
weave_pattern = Image.open('result_images/weave_pattern.png')
# call the function
processed_image = process_weave_pattern(weave_pattern)

5


In [25]:
## below is the logic for above code

In [20]:
# from PIL import Image

# def calculate_thread_parameters(image_path):
#     img = Image.open(image_path).convert('RGBA')

#     thread_color = (255, 0, 0, 255)  # Thread color
#     background_color = (255, 255, 255, 255)  # Background color

#     thread_width_sum = 0
#     thread_width_count = 0

#     thread_distances = []  # Store distances between adjacent threads

#     in_thread = False
#     thread_start = None

#     for x in range(img.width):
#         thread_pixel = img.getpixel((x, 0))

#         if thread_pixel == thread_color:
#             if not in_thread:
#                 thread_start = x
#                 in_thread = True
#         else:
#             if in_thread:
#                 thread_end = x - 1
#                 thread_width = thread_end - thread_start 
#                 thread_width_sum += thread_width
#                 thread_width_count += 1

#                 if thread_start is not None:
#                     thread_distances.append(thread_start)  # Store distance between adjacent threads

#                 in_thread = False
#     average_thread_width = thread_width_sum / thread_width_count
#     average_thread_distance = thread_distances[1] - thread_distances[0] - average_thread_width

#     return average_thread_width, average_thread_distance

# image_path = 'result_images/weave_pattern.png'
# thread_width, thread_distance = calculate_thread_parameters(image_path)

# print(f"Estimated thread width: {thread_width}")
# print(f"Estimated thread distance: {thread_distance}")


[300, 600, 900, 1200, 1500]
Estimated thread width: 100.0
Estimated thread distance: 200.0
